<a href="https://colab.research.google.com/github/Fabiorsi1979/DS-Comunidade/blob/main/SQL_avan%C3%A7ado_CASE_WHEN_IIF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A clause SELECT

#### A clause SELECT é responsável em selecionar colunas, criar novas colunas através de operações matemáticas ou colunas criadas com condicionais.

#### Selecionar colunas
~~~SQL
    SELECT
      c.customer_id,
      c.customer_city,
      c.customer_state
    FROM customer c
~~~
#### Criar nova coluna através de operações matemáticas
~~~SQL
    SELECT oi.price,
       oi.freght_value,
       oi.price + (0.10 * oi.price) AS valor_c_juros
    FROM order_items oi
~~~
#### Colunas criadas por condicionais
~~~SQL
    SELECT
      oi.price,
      CASE
          WHEN oi.price < 20 THEN 'barato' ELSE 'caro' END AS Status
    FROM order_items oi

    # aceita somente no SQLite
    SELECT
      IIF (oi.price < 20, 'barato', 'caro') AS Status
    FROM order_items oi
~~~
#### CASE WHEN ANINHADA
~~~SQL
    SELECT
      oi.price,
      CASE
          WHEN oi.price < 20 THEN 'super_barato'
          WHEN oi.price <100 THEN 'barato'
          WHEN oi.price > 150 AND oi.price <180 THEN 'normal'
          ELSE 'caro'
      END AS Status
    FROM order_items oi
~~~

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [8]:
path = '/content/drive/MyDrive/db_olist.sqlite'

In [9]:
import sqlite3
import pandas as pd

In [10]:
def Case_When_IFF (*args):
    conexao= sqlite3.connect(path)
    cursor = conexao.cursor()

    cursor.execute(consulta)
    dados = cursor.fetchall()

    cursor.close()
    conexao.cursor()

    consulta_sql = pd.DataFrame(data = dados)

    return consulta_sql

#### 1. Crie uma consulta que exiba o código do produto e a categoria de cada  produto com base no seu preço:
##### - Preço abaixo de 50 → Categoria A
##### - Preço entre 50 e 100 → Categoria B
##### - Preço entre 100 e 500 → Categoria C
##### - Preço entre 500 e 1500 → Categoria D
##### - Preço acima de 1500 → Categoria E

In [11]:
consulta ="""
SELECT
  oi.product_id,
  CASE
      WHEN oi.price < 50 THEN 'Categoria A'
      WHEN oi.price BETWEEN 50 AND 100 THEN 'Categoria B'
      WHEN oi.price BETWEEN 100 AND 500 THEN 'Categoria C'
      WHEN oi.price BETWEEN 500 AND 1500 THEN 'Categoria D'
      ELSE 'Categoria E'
  END AS Categoria_price
FROM order_items oi
LIMIT 10
"""
resultado = Case_When_IFF(consulta)
resultado.columns=["Código Produto","Categoria BASE Preço"]
resultado

,Código Produto,Categoria BASE Preço
0,4244733e06e7ecb4970a6e2683c13e61,Categoria B
1,e5f2d52b802189ee658865ca93d83a8f,Categoria C
2,c777355d18b72b67abbeef9df44fd0fd,Categoria C
3,7634da152a4610f1595efa32f14722fc,Categoria A
4,ac6c3623068f30de03045865e4e10089,Categoria C
5,ef92defde845ab8450f9d70c526ef70f,Categoria A
6,8d4f2bb7e93e6710a28f34fa83ee7d28,Categoria A
7,557d850972a7d6f792fd18ae1400d9b6,Categoria D
8,310ae3c140ff94b03219ad0adc3c778f,Categoria C
9,4535b0e1091c278dfd193e5a1d63b39f,Categoria B


 #### 2. Calcule a quantidade de produtos para cada uma das categorias criadas no exercícios anterior

In [12]:
consulta ="""
SELECT
  CASE
      WHEN oi.price < 50 THEN 'Categoria A'
      WHEN oi.price BETWEEN 50 AND 100 THEN 'Categoria B'
      WHEN oi.price BETWEEN 100 AND 500 THEN 'Categoria C'
      WHEN oi.price BETWEEN 500 AND 1500 THEN 'Categoria D'
      ELSE 'Categoria E'
  END AS Categoria_price,
  COUNT (DISTINCT oi.product_id)
FROM order_items oi
GROUP BY Categoria_price
LIMIT 10
"""
resultado = Case_When_IFF(consulta)
resultado.columns=["Categoria BASE Preço","Qtde de Produtos"]
resultado

,Categoria BASE Preço,Qtde de Produtos
0,Categoria A,11450
1,Categoria B,9087
2,Categoria C,11709
3,Categoria D,1335
4,Categoria E,205


#### 3. Selecione os seguintes categorias de produtos: livros técnicos, pet shop, pc gamer, tablets impressão imagem, fashion esports, perfumaria, telefonia, beleza saude, ferramentas jardim.
###### Crie uma primeira coluna mostrando o novo preço da categoria, segundo os valores abaixo:
###### - Livros técnicos - 10% de desconto
###### - Pet shop - 20% de desconto
###### - PC gamer - 50% de aumento
###### - Tablets - 10% de aumento
###### - Fashion Esports - 5% de aumento
###### - Crie uma segunda coluna mostrando se a categoria sofreu ou não alteração de preço


In [13]:
consulta ="""
SELECT
  p.product_category_name AS Categoria_filtro
FROM products p
WHERE p.product_category_name IN ("livros_tecnicos","pet_shop","pc_gamer","tablets_impressao_imagem","fashion_esporte","perfumaria","telefonia","beleza_saude","ferramentas_jardim")
GROUP BY p.product_category_name
"""
resultado = Case_When_IFF(consulta)
resultado.columns = ["Categorias"]
resultado

,Categorias
0,beleza_saude
1,fashion_esporte
2,ferramentas_jardim
3,livros_tecnicos
4,pc_gamer
5,perfumaria
6,pet_shop
7,tablets_impressao_imagem
8,telefonia


In [14]:
consulta ="""
SELECT
    p.product_category_name AS Categoria_reajuste,
    CASE
        WHEN p.product_category_name = "livros_tecnicos"          THEN (-oi.price*0.10 + oi.price)
        WHEN p.product_category_name = "pet_shop"                 THEN (-oi.price*0.20 + oi.price)
        WHEN p.product_category_name = "pc_gamer"                 THEN (oi.price*0.50 + oi.price)
        WHEN p.product_category_name = "tablets_impressao_imagem" THEN (oi.price*0.10 + oi.price)
        WHEN p.product_category_name = "fashion_esporte"          THEN (oi.price*0.05 + oi.price)
        ELSE oi.price
    END AS Reajuste
FROM products p LEFT JOIN order_items oi ON ( oi.product_id = p.product_id )
WHERE p.product_category_name IN (SELECT
                                      p.product_category_name AS Categoria
                                  FROM products p
                                  WHERE p.product_category_name IN ("livros_tecnicos","pet_shop","pc_gamer","tablets_impressao_imagem","fashion_esporte","perfumaria","telefonia","beleza_saude","ferramentas_jardim")
                                  GROUP BY p.product_category_name)
GROUP BY p.product_category_name
"""
resultado = Case_When_IFF(consulta)
resultado.columns = ["Categorias","Novo Preço "]
resultado

,Categorias,Novo Preço
0,beleza_saude,29.900
1,fashion_esporte,94.395
2,ferramentas_jardim,199.000
3,livros_tecnicos,44.091
4,pc_gamer,194.985
5,perfumaria,10.910
6,pet_shop,47.200
7,tablets_impressao_imagem,98.890
8,telefonia,25.990


In [15]:
consulta = """
SELECT
    tabela1.categoria AS Categoria_status,
    CASE
        WHEN (tabela1.Categoria = "livros_tecnicos" OR tabela1.Categoria = "pet_shop") THEN "Desconto"
        WHEN (tabela1.Categoria = "pc_gamer" OR tabela1.Categoria = "tablets_impressao_imagem" OR tabela1.Categoria = "fashion_esporte"  ) THEN "Acrescimo"
        ELSE "Manteve preço"
    END AS Status
FROM (SELECT
  p.product_category_name AS Categoria
FROM products p
WHERE p.product_category_name IN ("livros_tecnicos","pet_shop","pc_gamer","tablets_impressao_imagem","fashion_esporte","perfumaria","telefonia","beleza_saude","ferramentas_jardim")
GROUP BY p.product_category_name) AS tabela1
"""
resultado = Case_When_IFF(consulta)
resultado

,0,1
0,beleza_saude,Manteve preço
1,fashion_esporte,Acrescimo
2,ferramentas_jardim,Manteve preço
3,livros_tecnicos,Desconto
4,pc_gamer,Acrescimo
5,perfumaria,Manteve preço
6,pet_shop,Desconto
7,tablets_impressao_imagem,Acrescimo
8,telefonia,Manteve preço


In [17]:
consulta ="""
SELECT
    tab1.Categoria_filtro,
    tab2.Reajuste,
    tab3.Status
FROM (SELECT
          p.product_category_name AS Categoria_filtro
      FROM products p
      WHERE p.product_category_name IN ("livros_tecnicos","pet_shop","pc_gamer","tablets_impressao_imagem","fashion_esporte","perfumaria","telefonia","beleza_saude","ferramentas_jardim")
      GROUP BY p.product_category_name) AS tab1 LEFT JOIN (SELECT
                                                            p.product_category_name AS Categoria_reajuste,
                                                            CASE
                                                                WHEN p.product_category_name = "livros_tecnicos"          THEN (-oi.price*0.10 + oi.price)
                                                                WHEN p.product_category_name = "pet_shop"                 THEN (-oi.price*0.20 + oi.price)
                                                                WHEN p.product_category_name = "pc_gamer"                 THEN (oi.price*0.50 + oi.price)
                                                                WHEN p.product_category_name = "tablets_impressao_imagem" THEN (oi.price*0.10 + oi.price)
                                                                WHEN p.product_category_name = "fashion_esporte"          THEN (oi.price*0.05 + oi.price)
                                                                ELSE oi.price
                                                            END AS Reajuste
                                                         FROM products p LEFT JOIN order_items oi ON ( oi.product_id = p.product_id )
                                                         WHERE p.product_category_name IN (SELECT
                                                                                              p.product_category_name AS Categoria
                                                                                           FROM products p
                                                                                           WHERE p.product_category_name IN ("livros_tecnicos","pet_shop","pc_gamer","tablets_impressao_imagem","fashion_esporte","perfumaria","telefonia","beleza_saude","ferramentas_jardim")
                                                                                           GROUP BY p.product_category_name)
GROUP BY p.product_category_name) AS tab2 ON ( tab2.Categoria_reajuste = tab1.Categoria_filtro )
                                              LEFT JOIN (SELECT
                                                              tabela1.categoria AS Categoria_status,
                                                              CASE
                                                                  WHEN (tabela1.Categoria = "livros_tecnicos" OR tabela1.Categoria = "pet_shop") THEN "Desconto"
                                                                  WHEN (tabela1.Categoria = "pc_gamer" OR tabela1.Categoria = "tablets_impressao_imagem" OR tabela1.Categoria = "fashion_esporte"  ) THEN "Acrescimo"
                                                                  ELSE "Manteve preço"
                                                              END AS Status
                                                         FROM (SELECT
                                                                  p.product_category_name AS Categoria
                                                               FROM products p
                                                               WHERE p.product_category_name IN ("livros_tecnicos","pet_shop","pc_gamer","tablets_impressao_imagem","fashion_esporte","perfumaria","telefonia","beleza_saude","ferramentas_jardim")
                                                         GROUP BY p.product_category_name) AS tabela1) AS tab3 ON ( tab3.Categoria_status = tab1.Categoria_filtro )
"""
resultado = Case_When_IFF(consulta)
resultado.columns = ["Categoria","Novo Preço","Status"]
resultado

,Categoria,Novo Preço,Status
0,beleza_saude,29.900,Manteve preço
1,fashion_esporte,94.395,Acrescimo
2,ferramentas_jardim,199.000,Manteve preço
3,livros_tecnicos,44.091,Desconto
4,pc_gamer,194.985,Acrescimo
5,perfumaria,10.910,Manteve preço
6,pet_shop,47.200,Desconto
7,tablets_impressao_imagem,98.890,Acrescimo
8,telefonia,25.990,Manteve preço


#### Mesmo exercício que o anterior só que com menos linhas

In [18]:
consulta ="""
SELECT
    tab1.Categoria,
    tab1.Reajuste,
    CASE
        WHEN (tab1.Categoria = "livros_tecnicos" OR tab1.Categoria = "pet_shop") THEN "Acrescimo"
        WHEN (tab1.Categoria = "pc_gamer" OR tab1.Categoria = "tablets_impressao_imagem" OR tab1.Categoria = "pfashion_esporte"  ) THEN "Desconto"
        ELSE "Manteve preço"
    END AS Status
FROM (SELECT
         p.product_category_name AS Categoria,
         CASE
             WHEN p.product_category_name = "livros_tecnicos"          THEN (-oi.price*0.10 + oi.price)
             WHEN p.product_category_name = "pet_shop"                 THEN (-oi.price*0.20 + oi.price)
             WHEN p.product_category_name = "pc_gamer"                 THEN (oi.price*0.50 + oi.price)
             WHEN p.product_category_name = "tablets_impressao_imagem" THEN (oi.price*0.10 + oi.price)
             WHEN p.product_category_name = "fashion_esporte"          THEN (oi.price*0.05 + oi.price)
             ELSE oi.price
         END AS Reajuste
      FROM products p LEFT JOIN order_items oi ON ( oi.product_id = p.product_id )
      WHERE p.product_category_name IN (SELECT
                                           p.product_category_name AS Categorias
                                        FROM products p
                                        WHERE p.product_category_name IN ("livros_tecnicos","pet_shop","pc_gamer","tablets_impressao_imagem","fashion_esporte","perfumaria","telefonia","beleza_saude","ferramentas_jardim")
                                        GROUP BY p.product_category_name)
      GROUP BY p.product_category_name) AS tab1
"""
resultado = Case_When_IFF(consulta)
resultado.columns = ["Categoria","Novo Preço","Status"]
resultado

,Categoria,Novo Preço,Status
0,beleza_saude,29.900,Manteve preço
1,fashion_esporte,94.395,Manteve preço
2,ferramentas_jardim,199.000,Manteve preço
3,livros_tecnicos,44.091,Acrescimo
4,pc_gamer,194.985,Desconto
5,perfumaria,10.910,Manteve preço
6,pet_shop,47.200,Acrescimo
7,tablets_impressao_imagem,98.890,Desconto
8,telefonia,25.990,Manteve preço


#### 4. Crie uma coluna que mostra o status de entrega do pedido. Se a coluna estiver vazia, o status é de pendente.
##### - “order_delivered_customer_date” - pendente
##### - “order_delivered_customer_date” menor que o dia 01 de Junho de 2017 - entregue
##### - “order_delivered_customer_date” maior que o dia 01 de Junho de 2017 - programado

In [19]:
consulta = """
SELECT
    DATE(o.order_delivered_customer_date),
    CASE
        WHEN o.order_delivered_customer_date IS NULL         THEN 'Pendente'
        WHEN o.order_delivered_customer_date <= '2017-06-01' THEN 'entregue'
        ELSE 'programado'
    END
FROM orders o
LIMIT 10
"""
resultado = Case_When_IFF(consulta)
resultado.columns=["Data","Status"]
resultado

,Data,Status
0,2017-10-10,programado
1,2018-08-07,programado
2,2018-08-17,programado
3,2017-12-02,programado
4,2018-02-16,programado
5,2017-07-26,programado
6,None,Pendente
7,2017-05-26,entregue
8,2017-02-02,entregue
9,2017-08-16,programado
